# core

> Fill in a module description here

In [1]:
#| default_exp core

In [5]:
#| hide
from nbdev.showdoc import *

In [6]:
#| export
import toml
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
reproduce_dir = os.getenv("REPROWORKDIR", Path("./reproduce"))
dev_image_tag = os.getenv("REPRODEVIMAGE")



def generate_config():
    with open(Path(reproduce_dir, 'config.toml'), 'w') as f:
        f.write(f'''
[authors]
author1.name = "Alex P. Miller"
author1.affiliation = "USC Marshall School of Business"
author1.email = "alex.miller@marshall.usc.edu"

# reproduce.work configuration
[repro]
version = "reproduce.work/v1/default"
# Stages in the build/publish process
stages = ["init", "develop", "build"]


[repro.files]
input = "document/main.md"
dynamic = "document/data.toml" # "dynamic" files are files that are generated by a script or the output of a programmatic analysis
latex_template = "document/latex/template.tex"
output = "document/latex/compiled.tex"
watch = ["document/main.md", "document/data.toml", "document/latex/template.tex"]

[repro.stage.init]
script = """
docker build -t {dev_image_tag} .
docker build -t tinytex {reproduce_dir}/Dockerfile.tinytex
docker build -t watcher {reproduce_dir}/Dockerfile.watch
"""

[repro.stage.develop]
script = """
docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=''
\INSERT{watch_cmd_here}
"""

[repro.stage.build]
script = """
docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \INSERT{var} in `input` file
docker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex
docker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex
"""
''')
        

    # By default, TOML files cannot reference variables defined earlier in the file
    # This is a hack to get around that which allows me to read in the TOML data,
    # and then backfill the watch command with the files listed in the TOML file itself.
    if '\INSERT{watch_cmd_here}' in open(Path(reproduce_dir, 'config.toml'), 'r').read():
        with open(Path(reproduce_dir, 'config.toml'), 'r') as f:
            base_config = toml.load(f)
        base_config.repro.stage.develop.replace(
            '\INSERT{watch_cmd_here}', 
            f"""docker run watcher "{','.join(base_config.repro.files.watch)}" "echo \"File has changed!\" && build_cmd" """
        )
        with open(Path(reproduce_dir, 'config.toml'), 'w') as f:
            f.write(toml.dumps(base_config))



def validate_base_config(base_config):
    required_keys = ['authors', 'repro', 'repro.files', 'repro.stages']
    for key in required_keys:
        if key not in base_config:
            return False
        if key=='repro.stages':
            for stage in base_config[key]:
                if stage not in base_config['repro']['stage']:
                    return False
    return True

# Write some basic tests
def test_validate_base_config():
    # Test a valid base_config
    base_config = {
        'authors': {
            'author1.name': 'Alex P. Miller',
            'author1.affiliation': 'USC Marshall School of Business',
            'author1.email': 'alex.miller@marshall.usc.edu'
        },
        'repro': {
            'version': 'reproduce.work/v1/default',
            'stages': ['init', 'develop', 'build']
        },
        'repro.files': {
            'input': 'document/main.md',
            'dynamic': 'document/data.toml',
            'latex_template': 'document/latex/template.tex',
            'output': 'document/latex/compiled.tex',
            'watch': ['document/main.md', 'document/data.toml', 'document/latex/template.tex']
        },
        'repro.stage.init': {
            'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
        },
        'repro.stage.develop': {
            'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
        },
        'repro.stage.build': {
            'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
        }
    }
    assert validate_base_config(base_config) == True
    
    # Test an invalid base_config
    base_config = {
        'authors': {
            'author1.name': 'Alex P. Miller',
            'author1.affiliation': 'USC Marshall School of Business',
            'author1.email': 'alex.miller@marshall.usc.edu'
        },
        'repro': {
            'version': 'reproduce.work/v1/default',
            'stages': ['init', 'develop', 'build']
        },
        'repro.files': {
            'input': 'document/main.md',
            'dynamic': 'document/data.toml',
            'latex_template': 'document/latex/template.tex',
            'output': 'document/latex/compiled.tex'
        },
        'repro.stage.init': {
            'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
        },
        'repro.stage.develop': {
            'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
        },
        'repro.stage.build': {
            'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
        }
    }
    assert validate_base_config(base_config) == False
    
    print("All tests passed!")

def read_base_config():
    with open(Path(reproduce_dir, 'config.toml'), 'r') as f:
        base_config = toml.load(f)
    return base_config

In [13]:
#import os
#os.listdir(os.getcwd())

In [14]:
#| hide
import nbdev; nbdev.nbdev_export()